<a href="https://colab.research.google.com/github/Sireesha024/CS59000_06_Ethical_Artificial_Intelligence/blob/main/LSTM_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thovylnguyn/fakenewscorpus-full")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fakenewscorpus-full


In [3]:
import pandas as pd
df = pd.read_csv("/kaggle/input/fakenewscorpus-full/news_cleaned_big.csv")

<ipython-input-3-9a1aa3755189>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/fakenewscorpus-full/news_cleaned_big.csv")


In [4]:
df.head()

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN


In [5]:
columns_to_drop = [
    'Unnamed: 0', 'id', 'url', 'scraped_at', 'inserted_at', 'updated_at', 'meta_keywords', 'source', 'authors', 'keywords','meta_description', 'tags','summary'
]

In [6]:
df = df.drop(columns=columns_to_drop)


In [7]:
df.head()

,domain,type,content,title
0,express.co.uk,rumor,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...
1,barenakedislam.com,hate,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump
2,barenakedislam.com,hate,The Los Angeles Police Department has been den...,Donald Trump
3,barenakedislam.com,hate,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB..."
4,barenakedislam.com,hate,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will..."


In [8]:
unique_targets = df['type'].unique()
unique_targets

array(['rumor', 'hate', 'unreliable', 'conspiracy', 'clickbait', 'satire',
       'fake', 'reliable', 'bias', 'political', 'junksci', nan, 'unknown',
       '2018-02-02 01:19:41.756664', '2018-02-07 23:39:33.852696',
       '2017-11-10T11:18:44.524042', '2018-02-08 19:18:34.468066',
       ' including Iraq', '2018-02-08 19:18:34.468038',
       '” said Linton Roberts', '2018-02-10 13:43:39.521661'],
      dtype=object)

In [9]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00


In [10]:
#lstm

import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Step 1: Set your valid classes
valid_types = ['reliable', 'bias', 'fake', 'conspiracy', 'unreliable', 'clickbait']

# Step 2: Filter and balance dataset
# df must have columns: 'domain', 'title', 'content', 'type'
df_filtered = df[df['type'].isin(valid_types)].copy()

# Drop rows with missing domain, title, or content
df_filtered.dropna(subset=['domain', 'title', 'content'], inplace=True)
df_filtered.reset_index(drop=True, inplace=True)

print(df_filtered['type'].value_counts())


type
reliable      1913205
bias          1138658
fake           894736
conspiracy     759165
unreliable     298583
clickbait      231949
Name: count, dtype: int64


In [11]:
print("Number of NaN rows after dropping:", df_filtered.isna().sum().sum())

Number of NaN rows after dropping: 0


In [12]:
df_filtered.head()
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5236296 entries, 0 to 5236295
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   domain   object
 1   type     object
 2   content  object
 3   title    object
dtypes: object(4)
memory usage: 159.8+ MB


In [13]:
df_filtered.head()

,domain,type,content,title
0,blackagendareport.com,unreliable,UN-Backed Police Massacred Haitians With Impun...,UN-Backed Police Massacred Haitians With Impunity
1,blackagendareport.com,unreliable,It should have come as no surprise that the sa...,Black Agenda Report
2,blackagendareport.com,unreliable,"“When the police finally left the campus, arou...",Black Agenda Report
3,blackagendareport.com,unreliable,Zambia Must Clarify Whether It Will Host Israe...,Zambia Must Clarify Whether It Will Host Israe...
4,blackgenocide.org,conspiracy,\n\n\n\n\n\n\n\nRev Dr. Childress is available...,BlackGenocide.org


In [14]:
# Using len() to count the rows
num_rows = len(df_filtered)

# Or using the .shape attribute
num_rows = df_filtered.shape[0]

print(f'The number of rows is: {num_rows}')


The number of rows is: 5236296


In [15]:

samples_per_class = 50000
df_balanced = df_filtered.groupby('type', group_keys=False).apply(
    lambda x: x.sample(n=min(samples_per_class, len(x)), random_state=42)
).reset_index(drop=True)

# Step 3: Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_balanced['type'])

# Step 4: Create input text by combining fields
texts = (
    df_balanced['domain'].fillna('') + ' ' +
    df_balanced['title'].fillna('') + ' ' +
    df_balanced['content'].fillna('')
)

# Step 5: Tokenize and pad
tokenizer = Tokenizer(num_words=50000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=300)

# Step 6: Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: TPU support or fallback
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

# Step 8: Build and train LSTM model
with strategy.scope():
    model = Sequential([
        Embedding(50000, 128, input_length=300),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=3, batch_size=256, validation_data=(X_test, y_test), verbose=1)

# Step 9: Evaluate model
y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)
print(classification_report(
    y_test, y_pred,
    labels=range(len(label_encoder.classes_)),
    target_names=label_encoder.classes_,
    zero_division=0
))


<ipython-input-15-204d68abf8f7>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_filtered.groupby('type', group_keys=False).apply(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 1318s 1s/step - accuracy: 0.6707 - loss: 0.9385 - val_accuracy: 0.8356 - val_loss: 0.5113
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 1292s 1s/step - accuracy: 0.8414 - loss: 0.4875 - val_accuracy: 0.7867 - val_loss: 0.6026
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 1291s 1s/step - accuracy: 0.8384 - loss: 0.4749 - val_accuracy: 0.8795 - val_loss: 0.3665
              precision    recall  f1-score   support

        bias       0.82      0.81      0.82     10009
   clickbait       0.83      0.89      0.86      9918
  conspiracy       0.84      0.89      0.87      9980
        fake       0.95      0.87      0.91     10035
    reliable       0.93      0.93      0.93     10136
  unreliable       0.92      0.88      0.90      9922

    accuracy                           0.88     60000
   macro avg       0.88      0.88      0.88     60000
weighted avg       0.88      0.88      0.88     60000



In [16]:

# Step 10: Save model and encoders
model.save("lstm_balanced_model.h5")
with open("lstm_label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
with open("lstm_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Model training and saving complete.")


Model training and saving complete.
